## Standard errors

This is a notebook demonstrating how to obtain standard errors for your generated impact estimates.

In [ ]:
%load_ext autoreload
%autoreload 2
import os, sys
import warnings
warnings.filterwarnings('ignore') # suppress sklearn deprecation warnings for now..

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# the below checks for whether we run dowhy, causaltune, and FLAML from source
root_path = root_path = os.path.realpath('../..')
try:
    import causaltune
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "auto-causality"))

try:
    import dowhy
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "dowhy"))

try:
    import flaml
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "FLAML"))


In [ ]:
# this makes the notebook expand to full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%%javascript

// turn off scrollable windows for large output
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
from causaltune import CausalTune
from causaltune.datasets import synth_ihdp

## Loading data

In [ ]:
# load toy dataset and apply standard pre-processing
cd = synth_ihdp()
cd.preprocess_dataset()

In [ ]:
# inspect the preprocessed dataset
display(cd.data.head())

## Model training and standard errors

In [ ]:
# training configs

# choose estimators of interest
estimator_list = [

            # "Dummy",
            # "SparseLinearDML",
            # "ForestDRLearner",
            # "TransformedOutcome",
            # "CausalForestDML",
            ".LinearDML",
            # "DomainAdaptationLearner",
            "SLearner",
            "XLearner",
            # "TLearner",
            # "Ortho"
    ]

# set evaluation metric
metric = "energy_distance"

# it's best to specify either time_budget or components_time_budget, 
# and let the other one be inferred; time in seconds
time_budget = None
components_time_budget = 10

# specify training set size
train_size = 0.7


In [ ]:
ct = CausalTune(
    estimator_list=estimator_list,
    metric=metric,
    verbose=3,
    components_verbose=2,
    time_budget=time_budget,
    components_time_budget=components_time_budget,
    train_size=train_size
)


# run causaltune
ct.fit(data=cd, outcome=cd.outcomes[0])

In [ ]:
# obtaining effect estimates
cates = ct.effect(ct.test_df)
display(cates)

Below we show how to generate standard errors using `causaltune`. By default, this will use the `best_estimator` identified during training.

If this estimator does not have analytical standard errors, it will be refitted `n_bootstrap_samples` times on the training data.

In [ ]:
# generating standard errors by refitting train_df 
se = ct.effect_stderr(ct.test_df)
display(se)